In [1]:
!pip install bs4
!pip install sqlalchemy
!pip install requests
!pip install pandas
!pip install firecrawl 
!pip install psycopg2

In [1]:
from bs4 import BeautifulSoup
from firecrawl import FirecrawlApp
app = FirecrawlApp(api_key='fc-e6d03fc937144bc2a873053cfecd5987')
target_url = 'https://www.indeed.com/jobs?q=financial+analyst&l=&vjk=5484793a522f9eed'

# Scrape the raw HTML
scrape_result = app.scrape_url(target_url, formats=['html'])



In [18]:
import re
# Access the HTML content from the response object
html_content = scrape_result.html  # ✅ This is the correct attribute

# Parse with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find the first job listing
# get all jobs from the page

job_cards = soup.select('.resultContent')


def extract_average_salary(salary_str):
    try:
        # Find all numbers in the string (removes commas)
        numbers = re.findall(r'\d[\d,]*', salary_str)
        salaries = [int(num.replace(',', '')) for num in numbers]

        if len(salaries) == 0:
            return -1
        elif len(salaries) == 1:
            return salaries[0]
        else:
            return sum(salaries[:2]) // 2  # Only average the first two numbers
    except:
        return -1

i = 0
all_jobs = []
# Extract data
for job in job_cards:
    company_location = job.select_one('div.company_location')
    job_title = job.select_one('h2.jobTitle')
    salary = job.select_one('.salary-snippet-container')
    # Get the job title and link
    if job_title is not None:
        job_title_text = job_title.get_text(strip=True)
    else:
        job_title_text = "N/A"

    if salary is not None:
        job_salary_text = str(extract_average_salary(salary.get_text(strip=True)))
    else:
        job_salary_text = "65000"
    
    if company_location is not None:
        company_location_text = company_location.get_text(strip=True)
    else:
        company_location_text = "N/A"

    job = {
        'job_title': job_title_text,
        'job_salary': job_salary_text,
        'company_name_location': company_location_text
    }
    print(job)
    all_jobs.append(job)
    


{'job_title': 'Financial Systems Analyst', 'job_salary': '110000', 'company_name_location': 'California Dental AssociationRemote'}
{'job_title': 'Financial Analyst', 'job_salary': '70000', 'company_name_location': 'ReadyNineRemote'}
{'job_title': 'Financial Analyst', 'job_salary': '75617', 'company_name_location': 'NAVAC Inc.Lyndhurst, NJ 07071'}
{'job_title': 'Financial Analyst', 'job_salary': '92500', 'company_name_location': 'Advanced Medical ManageLong Beach, CA 90815'}
{'job_title': 'Financial Analyst', 'job_salary': '75000', 'company_name_location': 'Bunting Magnetics Co.DuBois, PA 15801'}
{'job_title': 'Financial Analyst', 'job_salary': '65000', 'company_name_location': 'Fareva USAHenrico, VA 23231'}
{'job_title': 'Financial Analyst', 'job_salary': '79800', 'company_name_location': 'Visiting Nurse Service of New York d/b/a VNS...Remote in New York, NY 10001'}
{'job_title': 'Senior Financial Analyst - Operations', 'job_salary': '130000', 'company_name_location': 'DivertRemote in 

In [20]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

# Load credentials from .env
load_dotenv()

# Database connection info
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

# Create SQLAlchemy engine
engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

# SQL statements
CREATE_TABLE_QUERY = """
CREATE TABLE IF NOT EXISTS indeed_jobs (
    id SERIAL PRIMARY KEY,
    job_title TEXT,
    job_salary TEXT,
    company_name_location TEXT
);
"""

INSERT_QUERY = """
INSERT INTO indeed_jobs (
    job_title, job_salary, company_name_location
)
VALUES (
    :job_title, :job_salary, :company_name_location
) 
RETURNING id;
"""



In [21]:

# Execute DB operations
try:
    with engine.begin() as conn:  # automatically commits or rolls back
        conn.execute(text(CREATE_TABLE_QUERY))
        
        for job in all_jobs:
            # Execute the insert statement
            conn.execute(text(INSERT_QUERY), job)

        print(f"Inserted {len(all_jobs)} job listings into the database.")
except SQLAlchemyError as e:
    print("Database error:", e)

Inserted 15 job listings into the database.
